In [1]:
import pandas as pd 
import os

In [2]:
area = pd.read_excel("Area.xlsx")
E = ['AH','BJ','CQ','FJ','GD','GS','GX','GZ','HaN','HeB','HeN','HLJ','HuB','HuN','JL','JS','JX','LN','NMG','NX','QH',
     'SaX','SC','SD','SH','SX','TJ','XJ','XZ','YN','ZJ']
Z = ['安徽','北京','重庆','福建','广东','甘肃','广西','贵州','海南','河北','河南','黑龙江','湖北','湖南','吉林','江苏','江西','辽宁','内蒙古','宁夏','青海',
     '陕西','四川','山东','上海','山西','天津','新疆','西藏','云南','浙江']

In [3]:
def pro_NTL_df(prov):
    p = os.getcwd()+"\\Pro_mean_DNB_BRDF_Corrected_NTL\\{}\\".format(prov)
    files = [p+f for f in os.listdir(p)]
    df=pd.DataFrame()
    for f in files:
        a = pd.read_csv(f)[['ind','UFZ']]
        a['Month']=f.replace(p,"").replace(".csv","").replace(prov,"")+"-01"
        df=df.append(a)
    df.columns=['BRDF_MEAN','UFZ','Month']
    df['P'] = prov
    df['Month'] = pd.to_datetime(df['Month'])
    df['year'] = df['Month'].dt.year
    return df

In [4]:
def NTL_FZ(df,prefix:str):
    '''
    Function: Compute the weighted NTL in each zone by their acreage
    return: dataframe: Month + NTL in that zone(s).
    '''
    df = pd.merge(df,df[['Month','P','Shape_Area']].groupby(['Month','P'],as_index=False).sum(),on=['Month','P'])
    df[prefix+'_BRDF'] = df['BRDF_MEAN'] * df['Shape_Area_x'] / df['Shape_Area_y'] 
    df = df[[prefix+'_BRDF','Month','P']].groupby(['Month','P'],as_index=False).sum()
    del df['P']
    df['Month'] = pd.to_datetime(df['Month'])
    return df

In [5]:
def NTL(area,i):
    '''
    Function: Merge NTL in each zone.
    return: dataframe
    '''    
    df = pro_NTL_df(E[i])
    df = pd.merge(df,area,on=['year','P','UFZ'])
    commercial = NTL_FZ(df[(df['UFZ']=='commercial') ],"C").set_index('Month')
    residential = NTL_FZ(df[(df['UFZ']=='residential') ],"R").set_index('Month')
    industrial = NTL_FZ(df[(df['UFZ']=='industrial') ],"I").set_index('Month')       
    amenity = NTL_FZ(df[(df['UFZ']=='amenity') ],"A").set_index('Month')    
    mix = NTL_FZ(df[(df['UFZ']=='mix') ],"M").set_index('Month')      
    inactive = NTL_FZ(df[(df['UFZ']=='-') | (df['UFZ']=='nature') ],"N").set_index('Month')
    active = NTL_FZ(df[(df['UFZ']!='-') & (df['UFZ']!='nature') ],"T").set_index('Month')
    overall = NTL_FZ(df,"O").set_index('Month')
    df = pd.concat([commercial,residential,industrial,amenity,mix,inactive,active,overall],axis=1)
    df['province'] = Z[i]
    return df

In [6]:
df = NTL(area,0)
df.to_csv("Pro_mean_DNB_BRDF_Corrected_NTL.csv",encoding='utf-8-sig')
for i in range(1,31):
    df = NTL(area,i)
    df.to_csv("Pro_mean_DNB_BRDF_Corrected_NTL.csv",encoding='utf-8-sig',header=None, mode='a')

ratio

In [7]:
def NTL_to_mon_mean(aa,prefix:str):
    '''
    Function: NTL(month)/mean(NTL,month). 
        Eg: NTL(2017-12)/mean(NTL(2017-12)+NTL(2018-12)+...+NTL(2022-12))*100
        Eg: NTL(2017-11)/mean(NTL(2017-11)+NTL(2018-11)+...+NTL(2022-11))*100
    return dataframe, 0<NTL_R<100
    '''
    dd = aa[[prefix+'_BRDF','province','m']].groupby(['province','m'],as_index=False).mean()
    dd.columns = ['province','m','BRDF_mon']
    aa = pd.merge(aa,dd,on=['province','m'],how='left')
    aa[prefix+'_BRDF_R'] = aa[prefix+'_BRDF']/aa['BRDF_mon']*100
    del aa['BRDF_mon']
    return aa 

In [8]:
df = pd.read_csv("Pro_mean_DNB_BRDF_Corrected_NTL.csv")
df['Month'] = pd.to_datetime(df['Month'])
df['m'] = df['Month'].dt.month

In [9]:
for prefix in ["C","R","I","A","M","N","T","O"]:
    df = NTL_to_mon_mean(df[df['Month']>pd.to_datetime('2016-12-31')],prefix)

In [10]:
df

,Month,C_BRDF,R_BRDF,I_BRDF,A_BRDF,M_BRDF,N_BRDF,T_BRDF,O_BRDF,province,m,C_BRDF_R,R_BRDF_R,I_BRDF_R,A_BRDF_R,M_BRDF_R,N_BRDF_R,T_BRDF_R,O_BRDF_R
0,2017-01-01,24.833136,12.381579,8.236132,18.036454,31.634457,1.241401,12.137514,1.519789,安徽,1,140.670872,96.068257,75.797200,168.904484,153.482633,84.795535,128.199520,78.945559
1,2017-02-01,23.030551,11.491643,7.822444,16.686533,28.925912,1.269474,11.329204,1.526493,安徽,2,133.920324,90.740149,72.535400,171.784373,144.311298,85.800352,125.246312,78.740006
2,2017-03-01,23.870619,11.943443,7.879352,17.335630,31.227733,1.213596,11.676171,1.480907,安徽,3,137.445129,94.534183,72.965863,174.516002,148.742029,86.673816,127.385494,79.704098
3,2017-04-01,24.296915,12.077707,8.390740,17.737078,33.343081,1.304028,12.012630,1.577625,安徽,4,139.628563,96.733175,77.138169,178.121605,152.833085,98.254519,131.654239,88.646322
4,2017-05-01,23.665730,11.656097,7.944923,17.292727,32.415799,1.185377,11.560699,1.450460,安徽,5,139.763020,96.074287,75.468583,180.593218,154.547094,89.633648,131.004688,82.198443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2227,2022-08-01,15.524182,14.963376,15.380272,25.962775,17.657734,2.823632,16.056076,3.974973,浙江,8,128.312736,101.174053,104.839529,101.182744,82.824459,99.220763,109.279794,112.017428
2228,2022-09-01,15.792486,14.984446,15.277861,26.331558,17.492134,2.991411,16.102230,4.132170,浙江,9,130.304260,100.905829,104.967846,101.375559,81.318246,102.698053,109.593906,114.340553
2229,2022-10-01,17.668756,17.076908,16.895071,30.036335,20.007616,3.373005,18.150646,4.658793,浙江,10,138.279944,107.388566,110.689922,108.443867,87.520268,110.082832,116.305038,122.223018
2230,2022-11-01,16.671921,15.730447,15.599882,27.672021,18.380862,3.353051,16.797296,4.522821,浙江,11,129.054121,98.429477,101.316294,99.157556,79.231464,109.055220,106.874889,118.379640


In [11]:
df.to_csv("Pro_ratio_DNB_BRDF_Corrected_NTL.csv",encoding='utf-8-sig',index=None)